In [19]:
import backtrader as bt
import yfinance as yf
from datetime import datetime

# ---- 1. Define the simplest strategy ----
class TestStrategy(bt.Strategy):
    def next(self):
        print(f"{self.data.datetime.date(0)} | Close: {self.data.close[0]}")

# ---- 2. Download data (single string ticker, not a tuple) ----
data = yf.download("AAPL", start="2020-01-01", end="2020-06-01")

# ---- 3. Wrap data for Backtrader ----
feed = bt.feeds.PandasData(dataname=data)

# ---- 4. Set up Cerebro ----
cerebro = bt.Cerebro()
cerebro.addstrategy(TestStrategy)   # no params, simplest
cerebro.adddata(feed)
cerebro.broker.setcash(10000)

# ---- 5. Run ----
print("Starting Portfolio Value:", cerebro.broker.getvalue())
cerebro.run()
print("Final Portfolio Value:", cerebro.broker.getvalue())


Starting Portfolio Value: 10000.0


AttributeError: 'tuple' object has no attribute 'lower'

Found 27 unique tickers


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AVGO']: Timeout('Failed to perform, curl: (28) Operation timed out after 10001 milliseconds with 0 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


✗ No data for AVGO
✓ Loaded AMZN from cache
✓ Loaded GOOGL from cache
✓ Loaded NVDA from cache
✓ Loaded TEM from cache
✓ Loaded VST from cache
✓ Loaded PANW from cache
✓ Loaded AAPL from cache
✓ Loaded MSFT from cache
✓ Loaded GOOG from cache
✓ Loaded TSLA from cache
✓ Loaded AXP from cache
✓ Loaded DIS from cache
✓ Loaded PYPL from cache
✓ Loaded MU from cache
✓ Loaded CRM from cache
✓ Loaded RBLX from cache
✓ Loaded AB from cache
✓ Loaded CRWD from cache
✓ Loaded NFLX from cache

Successfully loaded 19 tickers


/var/folders/v0/tllws09j6qnc4kds9pv97y6r0000gn/T/ipykernel_77109/2315372195.py:44: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  stock_data = pd.read_csv(cache_file, index_col=0, parse_dates=True)
/var/folders/v0/tllws09j6qnc4kds9pv97y6r0000gn/T/ipykernel_77109/2315372195.py:44: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  stock_data = pd.read_csv(cache_file, index_col=0, parse_dates=True)
/var/folders/v0/tllws09j6qnc4kds9pv97y6r0000gn/T/ipykernel_77109/2315372195.py:44: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  stock_data = pd.read_csv(cache_file, index_col=0, parse_d

In [13]:

print(f'\nStarting Portfolio Value: {cerebro.broker.getvalue():.2f}')
results = cerebro.run()
# print(f'Final Portfolio Value: {cerebro.broker.getvalue():.2f}')
# strat = results
# (print analysis as before...)
# cerebro.plot(style='candlestick')



Starting Portfolio Value: 100000.00


TypeError: must be real number, not str

In [ ]:
print(df.head())
print(df.columns)